**Assignment 3 PDP**

In [29]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import numpy as np
import pandas as pd

Initiate Spark

In [30]:
findspark.init()

Create a Spark session

In [31]:
spark = SparkSession.builder.master("local").appName("Assignment3").config("spark.executor.memory", "1gb").getOrCreate()
sc = spark.sparkContext

Load the data

In [32]:
df = spark.read.format("CSV").option("header", "true").load("titanic.csv")

Format the data

In [33]:
df = df.withColumn("Survived", df["Survived"].cast(IntegerType())) \
    .withColumn("Pclass", df["Pclass"].cast(IntegerType())) \
    .withColumn("Name", df["Name"].cast(StringType())) \
    .withColumn("Sex", df["Sex"].cast(StringType())) \
    .withColumn("Age", df["Age"].cast(IntegerType())) \
    .withColumn("Siblings/Spouses Aboard", df["Siblings/Spouses Aboard"].cast(IntegerType())) \
    .withColumn("Parents/Children Aboard", df["Parents/Children Aboard"].cast(IntegerType())) \
    .withColumn("Fare", df["Fare"].cast(FloatType()))

a.
In order to calculate the probability that the person survives the passengers have been grouped by gender and class

In [34]:
passenger_survival = df.select("Survived", "Sex", "Pclass").toPandas()
passenger_survival = passenger_survival.groupby(["Sex", "Pclass"]).mean(["Survived"])
passenger_survival

Survived
Sex    Pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.137026

b. All the children that are in third class and 10 yrs or younger are selected. Then the children that have survived are being counted. Finally, the amount of children that had survived is divided by the total number of children that satisfy the condition of being in third class and 10yrs or younger. 




In [35]:
survival_probability = df.select("Age", "Pclass", "Survived").toPandas()
survival_probability = survival_probability[(survival_probability['Age'] <= 10) & (survival_probability['Pclass'] == 3)][['Age', 'Survived', 'Pclass']]
survived = survival_probability[survival_probability['Survived'] == 1].count()[0]
chance = (survived / survival_probability.count()[0] * 100).item()
print("Chance of survival: " + str(chance))

Chance of survival: 41.509433962264154


c. The mean is being used to calculate the average ticket price per passenger class. 

In [36]:
avg_ticket_price = df.select("Pclass", "Fare").toPandas()
avg_ticket_price = avg_ticket_price.groupby(["Pclass"]).mean(["Fare"])
avg_ticket_price

,Fare
Pclass,
1,84.154686
2,20.662184
3,13.707707
